In [22]:
import tensorflow as tf
import numpy as np
import pandas as pd
import glob

In [23]:
def present_tensorboard_logs(stage='feature_selection'):
    # Create dict of values
    df_dict = {
        'run_name': [],
        'max_binary_accuracy': [],
        'max_binary_accuracy_epoch': [],
        'max_precision': [],
        'max_precision_epoch': [],
        'max_recall': [],
        'max_recall_epoch': [],
        'max_f1': [],
        'max_f1_epoch': []
    }

    if stage == 'feature_selection':
    # Collect the validation filepaths
        filepaths = glob.glob('./tensorboard/feature_selection/*/validation/*')
    elif stage == 'model_selection':
        filepaths = glob.glob('./tensorboard/model_testing/*/validation/*')
    elif stage == 'model_tuning':
        filepaths = glob.glob('./tensorboard/model_tuning/*/*/*/validation/*')

    # Iterate over the filepaths
    for filepath in filepaths:
        if stage == 'model_tuning':
            string_arr = run_name = filepath.split('/')
            run_name = f'{string_arr[3]}_{string_arr[4]}'
        else:
            run_name = filepath.split('/')[3]
        # Set the empty arrays
        epoch_binary_accuracy_arr = []
        epoch_precision_arr = []
        epoch_recall_arr = []
        epoch_f1_arr = []

        # Initialise tf iterator to iterate over the log file
        for event in tf.compat.v1.train.summary_iterator(filepath):
            for value in event.summary.value:
                if value.tag == 'epoch_binary_accuracy':
                    num = tf.make_ndarray(value.tensor).ravel()[0]
                    epoch_binary_accuracy_arr.append(num)
                elif value.tag == 'epoch_precision':
                    num = tf.make_ndarray(value.tensor).ravel()[0]
                    epoch_precision_arr.append(num)
                elif value.tag == 'epoch_recall':
                    num = tf.make_ndarray(value.tensor).ravel()[0]
                    epoch_recall_arr.append(num)

        # Compute max and epoch max
        epoch_binary_accuracy_max = np.max(epoch_binary_accuracy_arr)
        epoch_binary_accuracy_max_epoch = epoch_binary_accuracy_arr.index(epoch_binary_accuracy_max)
        epoch_precision_max = np.max(epoch_precision_arr)
        epoch_precision_max_epoch = epoch_precision_arr.index(epoch_precision_max)
        epoch_recall_max = np.max(epoch_recall_arr)
        epoch_recall_max_epoch = epoch_recall_arr.index(epoch_recall_max)
        
        # Compute F1 score for each epoch and select the max and the epoch it relates to
        for precision, recall in zip(epoch_precision_arr, epoch_recall_arr):
            f1_score = 2 * (precision * recall) / (precision + recall)
            if np.isnan(f1_score):
                epoch_f1_arr.append(0)
            else:
                epoch_f1_arr.append(f1_score)

        epoch_f1_max = np.max(epoch_f1_arr)
        epoch_f1_max_epoch = epoch_f1_arr.index(epoch_f1_max)
        
        # Update dict
        df_dict['run_name'].append(run_name)
        df_dict['max_binary_accuracy'].append(epoch_binary_accuracy_max)
        df_dict['max_binary_accuracy_epoch'].append(epoch_binary_accuracy_max_epoch)
        df_dict['max_precision'].append(epoch_precision_max)
        df_dict['max_precision_epoch'].append(epoch_precision_max_epoch)
        df_dict['max_recall'].append(epoch_recall_max)
        df_dict['max_recall_epoch'].append(epoch_recall_max_epoch)
        df_dict['max_f1'].append(epoch_f1_max)
        df_dict['max_f1_epoch'].append(epoch_f1_max_epoch)

    # Create dataframe
    df = pd.DataFrame.from_dict(df_dict)
    df.set_index('run_name', inplace=True)
    df.sort_values('max_f1', ascending=False, inplace=True)
    return df

In [24]:
present_tensorboard_logs()

,max_binary_accuracy,max_binary_accuracy_epoch,max_precision,max_precision_epoch,max_recall,max_recall_epoch,max_f1,max_f1_epoch
run_name,,,,,,,,
run_pairwisecorr__boruta_01-14-2024-14:16:04,0.698578,13,0.392012,13,0.885132,3,0.484584,17
run_boruta_01-14-2024-14:40:32,0.581991,4,0.329438,4,0.906438,1,0.468319,4
run_vif_01-14-2024-14:23:39,0.641601,28,0.342770,28,0.928208,0,0.466315,16
run_vif__boruta_01-14-2024-14:31:03,0.583992,34,0.324927,34,0.858268,4,0.457744,2
run_vif__umap_01-14-2024-14:50:45,0.656767,11,0.345592,11,0.781843,0,0.435312,9
run_pairwisecorr_01-14-2024-14:15:30,0.562085,0,0.274124,0,0.930523,9,0.403786,16
run_all_01-14-2024-14:14:58,0.706477,2,0.353408,2,0.963872,5,0.400226,18
run_boruta__umap_01-14-2024-14:45:33,0.674882,7,0.332054,9,0.907365,0,0.391881,1
run_pairwisecorr__boruta__umap_01-14-2024-14:19:45,0.718483,10,0.374512,10,0.358962,2,0.365049,2


In [25]:
present_tensorboard_logs('model_selection')

,max_binary_accuracy,max_binary_accuracy_epoch,max_precision,max_precision_epoch,max_recall,max_recall_epoch,max_f1,max_f1_epoch
run_name,,,,,,,,
baseline_24_hour,0.712251,7,0.405663,7,0.862117,5,0.493919,11
two_layer_dropout_24_hour,0.658436,18,0.372197,18,0.912256,1,0.493506,18
two_layer_12_hour,0.686268,22,0.388934,22,0.906858,1,0.490676,22
three_layer_dropout_12_hour,0.623353,11,0.353416,11,0.910565,9,0.488552,11
baseline_dropout_24_hour,0.688720,17,0.389321,17,0.896472,24,0.486957,17
two_layer_dropout_6_hour,0.642654,30,0.359836,30,0.935618,0,0.482853,30
three_layer_12_hour,0.641058,25,0.358053,25,0.933735,1,0.480317,25
baseline_12_hour,0.616503,16,0.345375,16,0.886932,7,0.476177,16
baseline_dropout_6_hour,0.612428,28,0.342449,28,0.900417,0,0.475862,20


In [26]:
present_tensorboard_logs('model_tuning').head(10)

,max_binary_accuracy,max_binary_accuracy_epoch,max_precision,max_precision_epoch,max_recall,max_recall_epoch,max_f1,max_f1_epoch
run_name,,,,,,,,
baseline_24_hour_0028,0.680384,12,0.387734,12,0.863510,4,0.499422,12
baseline_24_hour_0022,0.729239,4,0.419468,4,0.751625,0,0.490235,0
baseline_24_hour_0002,0.656220,1,0.367180,1,0.766945,0,0.483677,1
baseline_24_hour_0012,0.739897,1,0.435823,1,0.592386,0,0.481419,0
baseline_24_hour_0013,0.620977,1,0.349266,1,0.888115,0,0.481225,1
baseline_24_hour_0017,0.634695,5,0.354291,6,0.832869,0,0.480335,6
baseline_24_hour_0027,0.750448,1,0.449738,1,0.805014,10,0.476830,2
baseline_24_hour_0016,0.726918,7,0.419985,7,0.708449,4,0.476242,8
baseline_24_hour_0011,0.601456,0,0.339275,0,0.888115,1,0.475635,0
